### Contour Extraction and Handle Removal

#### Notes:

1. The code below finds the contours without handles of pots using three different methods depending on the pot style. It outputs an image of the pot with the contour imposed on top, as well as a CSV containing the coordiantes of the contours.
1. Requires three text files beforehand, one containing all the filenames of pots with no/one handle (alg 1), pots with bell-krater-like handles (alg 2) and pots with amphora-like handles (alg 3).
1. At this point, alg 1 doesn't require us to put down which side of the pot we want. It finds the shortest side since it assumes the shortest side is the side with no handle. This can be changed to specify which side we want by going to the code for alg 1 and changing the last variable in the function <i>get_contour_side</i> from "Z" to "L" or "R".
1. Alg 2 requires us to say where the pot handle roughly lies. At this point, it tries all three ranges of "top", "mid" and "bot" and works out the contour for all three. If you want to change this, simply remove the options you don't want in the code below.

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import pylab as pl
import pandas as pd
from math import sqrt
from skimage.filters import threshold_otsu
from skimage import measure
import os
import re
import itertools as it
import csv
from PIL import Image
from skimage.color import rgb2gray
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage import data, img_as_float
from skimage.segmentation import (morphological_chan_vese,
                                  morphological_geodesic_active_contour,
                                  inverse_gaussian_gradient,
                                  checkerboard_level_set)
pl.ion()

In [ ]:
f=open('C:\\Users\\arian\\Dropbox\\no_one_hnd.txt', "r")
no_one_hnd = f.readlines()
f=open('C:\\Users\\arian\\Dropbox\\bell_hnd.txt', "r")
bell_hnd = f.readlines()
f=open('C:\\Users\\arian\\Dropbox\\amph_hands.txt', "r")
amph_hnd = f.readlines()

In [ ]:
from ipynb.fs.full.contour_extraction_1 import cont_extraction_1
from ipynb.fs.full.contour_extraction_2 import cont_extraction_2
from ipynb.fs.full.contour_extraction_3 import cont_extraction_3

In [ ]:
direc_nme = 'C:\\Users\\arian\\Dropbox\\selected vase classes' # Folder with files to be renamed.
current_dir = 'C:\\Users\\arian\\Documents\\GitHub\\pots\\Code'

for filename in os.listdir(direc_nme): 
        
    A = re.search('\D+',filename).group(0)
    
    print("\nExtracting contours from "+A)

    newpath = r'C:\\Users\\arian\\Documents\\GitHub\\pots\\Code\\'+str(A)

    pot_class = A.replace(" ","-") #Some classes with multiple names have spaces in the folder name
    pot_class = pot_class.replace("_","-") # whilst others have underscores. We'll change them all to have dashes instead.

    if not os.path.exists(newpath):
        os.makedirs(newpath)

    for subdir, dirs, files in os.walk(direc_nme+"\\"+filename):
        for i in range(0,len(files)):

            pot_id = re.search('\d+',files[i]).group(0)
            pot_vers = re.search('(img).\d+',files[i])
            if (not pot_vers) == True:
                pot_vers = "img_0"
            else:
                pot_vers = pot_vers.group(0)


            if (str(files[i])+'\n' in no_one_hnd) == True:
                alg = "Alg 1"
            else:
                if (str(files[i])+'\n' in bell_hnd) == True:
                    alg = "Alg 2"
                else:
                    if (str(files[i])+'\n' in amph_hnd) == True:
                        alg = "Alg 3"
                    else:
                        alg = "no alg"

            pot_path = direc_nme+"\\"+filename+"\\"+str(files[i])


            if alg == "Alg 1":
                try:
                    image_grey,x,y = cont_extraction_1(pot_path,current_dir)

                    pot_nme = pot_class+"_"+str(pot_id)+"_"+str(pot_vers)

                    fig, ax = plt.subplots(figsize=(5, 7))
                    ax.imshow(image_grey,cmap='gray')
                    ax.plot(x,y,'-r')
                    ax.set_axis_off()
                    plt.savefig(current_dir+"\\"+str(A)+"\\"+pot_nme+".jpg")
                    plt.close()

                    cont_coord = {}
                    cont_coord['x'] = x
                    cont_coord['y'] = y
                    with open(current_dir+"\\"+str(A)+"\\"+pot_nme+".csv", "w", newline='') as outfile:
                        writer = csv.writer(outfile)
                        writer.writerow(cont_coord.keys())
                        writer.writerows(it.zip_longest(*cont_coord.values()))
                except:
                    print(pot_nme+" contour extraction unsuccesful")



            if alg == "Alg 2":

                positions = ["top","mid","bot"]
                for p in positions:
                    try:
                        image_grey,x,y = cont_extraction_2(pot_path,p)

                        pot_nme = pot_class+"_"+str(pot_id)+"_"+str(pot_vers)+"__"+p

                        fig, ax = plt.subplots(figsize=(5, 7))
                        ax.imshow(image_grey,cmap='gray')
                        ax.plot(x,y,'-r')
                        ax.set_axis_off()
                        plt.savefig(current_dir+'\\'+str(A)+"\\"+pot_nme+".jpg")
                        plt.close()

                        cont_coord = {}
                        cont_coord['x'] = x
                        cont_coord['y'] = y
                        with open(current_dir+"\\"+str(A)+"\\"+pot_nme+".csv", "w", newline='') as outfile:
                            writer = csv.writer(outfile)
                            writer.writerow(cont_coord.keys())
                            writer.writerows(it.zip_longest(*cont_coord.values()))
                    except:
                        print(pot_nme+" contour extraction unsuccesful")


            if alg == "Alg 3":

                try:
                    image_grey,x,y = cont_extraction_3(pot_path,current_dir)

                    pot_nme = pot_class+"_"+str(pot_id)+"_"+str(pot_vers)

                    fig, ax = plt.subplots(figsize=(5, 7))
                    ax.imshow(image_grey,cmap='gray')
                    ax.plot(x,y,'-r')
                    ax.set_axis_off()
                    plt.savefig(current_dir+"\\"+str(A)+"\\"+pot_nme+".jpg")
                    plt.close()

                    cont_coord = {}
                    cont_coord['x'] = x
                    cont_coord['y'] = y
                    with open(current_dir+"\\"+str(A)+"\\"+pot_nme+".csv", "w", newline='') as outfile:
                        writer = csv.writer(outfile)
                        writer.writerow(cont_coord.keys())
                        writer.writerows(it.zip_longest(*cont_coord.values()))
                except:
                    print(pot_nme+" contour extraction unsuccesful")
                        
                        